In [1]:
# import external libraries
import os
import sys
import random
import numpy as np
import pandas as pd
import torch
import math
import time

# import internal modules
sys.path.insert(1, '../src/')
from models.nn import MLP, TimeSeriesDataset
from utils.data_editor import lag, train_test_split

# No Batch

In [2]:
# read processed data
df = pd.read_csv("../data/processed/tidy_df.csv", index_col=[0, 1, 2])

# empty list for dataframes
y_test_list = []
y_hat_umlp = []

i = df.index.get_level_values(0).unique()[0]
print(i)

# y : "EPS"
y = df.loc[pd.IndexSlice[i, :, :], "EPS"]

# x, exogenous regressors : 'INV', 'AR', 'CAPX', 'GM', 'SA', 'ETR', 'LF'
#     x = df.loc[pd.IndexSlice[i, :, :], ['INV', 'AR', 'CAPX', 'GM', 'SA', 'ETR', 'LF']]

# Unlike statsmodel SARIMA package, NN needs to prepare lagged inputs manually if needed.
# y_lag and x_lag (lag 4 for now)
num_lag = 4
y_lag = lag(y, num_lag, drop_nan=False, reset_index=False)
#     x_lag = lag(x, num_lag, drop_nan=False, reset_index=False)

# Redefine data name as target (y) and feature (y_lag) (explanatory variable, predictor)
target = y
feature = y_lag

# save simple test data series
_, target_test_dataset = train_test_split(target, ratio=(4,1))
_, feature_test_dataset = train_test_split(feature, ratio=(4,1))

# drop nan caused by lag()
feature = feature.dropna(axis=0)
target = target[feature.index]

# setting torch
dtype = torch.float # double float problem in layer 
device = torch.device("cpu")

# Make data to torch.tensor
target = torch.tensor(target.values, dtype=dtype)
feature = torch.tensor(feature.values, dtype=dtype)
target_test_dataset = torch.tensor(target_test_dataset.values, dtype=dtype)
feature_test_dataset = torch.tensor(feature_test_dataset.values, dtype=dtype)

# rolling window data preparation

### ! Hyper-Parameter ! ##########################################################
# all period: 48, train 36, test 12
test_window = len(target_test_dataset)
print("test window: ", test_window)

train_window = len(target) - test_window
print("train window: ", train_window)
##################################################################################

train_dataset = TimeSeriesDataset(feature, target, train_window)
print("len of train dataset: ", len(train_dataset))
#     len(train_dataset) == len(target) - train_window = 48 - 36 = 12 == test_window
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)

np.random.seed(0)
torch.manual_seed(0)

### ! Hyper-Parameter ! ##########################################################
num_epochs = 500
learning_rate = 1e-2
input_units = 1
hidden_units = 512
# num_layers = 1
output_units = 1
# Optimizer

model_name = 'umlp' + '_hid' + str(hidden_units) + '_lr' + str(learning_rate) + '_epo' + str(num_epochs)
##################################################################################

# load rolling window data flow
for num_window, (feature_train, target_train) in enumerate(train_loader):
    print("rolling window: ", num_window)
    feature_train = feature_train[0] # extract single batch
    target_train = target_train[0] # extract single batch

#     #####Init the Model #######################
#     mlp = MLP(input_features=feature_train.size()[1], hidden_units=hidden_units, output_units=output_units)
#     ##### Set Criterion Optimzer and scheduler ####################
#     criterion = torch.nn.MSELoss(reduction="mean")
#     optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate) # link to mlp parameters (lr should be 1e-2)    
    
    #(only first window)
    if num_window == 0:
        #####Init the Model #######################
        mlp = MLP(input_features=feature_train.size()[1], hidden_units=hidden_units, output_units=output_units)
        ##### Set Criterion Optimzer and scheduler ####################
        criterion = torch.nn.MSELoss(reduction="mean")
        optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate) # link to mlp parameters (lr should be 1e-2)
    else:
        pass

    # Train the model: Learning iteration
    # use pre window trained model's weight as initial weight (continue training)
    #             print("initial weight: ", mlp.state_dict()['hidden.weight'][0])

    for step in range(num_epochs):
        # Forward pass
        target_pred = mlp(feature_train)
        # let y_pred be the same size as y
        target_pred = target_pred.squeeze(1)

        # Compute loss
        loss = criterion(target_pred, target_train) # link to mlp output
        if (step == 0) | (step == 1) | (step == 100) | (step == 499):
            print(f"step {step}: loss {loss.item()}")

        # Zero gradients, perform backward pass, and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Save the trained model
#     PATH = '../../assets/trained_models/univariate/mlp/' + model_name + "_" + i + "_" + "win" + str(num_window) + '.pth'
#     torch.save(mlp.state_dict(), PATH)
    # use the existing trained model and continute training next window.
    #             print("inherit weight: ", mlp.state_dict()['hidden.weight'][0])

    # predict y_hat (target_hat) <- 良くないかも、with torch_nograd() と model.eval()
    with torch.no_grad():
        target_test = target_test_dataset[num_window]
        feature_test = feature_test_dataset[num_window]
        y_hat_umlp.append(mlp(feature_test).squeeze().detach().numpy())
    #                 print(feature_test)
    #                 print(target_test)
    #                 print(y_hat_umlp[-1])

あらた
test window:  12
train window:  36
len of train dataset:  12
rolling window:  0
step 0: loss 2217.675537109375
step 1: loss 2070.275634765625
step 100: loss 69.08267211914062
step 499: loss 0.8574444651603699
rolling window:  1
step 0: loss 5.491274833679199
step 1: loss 23.569185256958008
step 100: loss 0.5542507767677307
step 499: loss 0.14426058530807495
rolling window:  2
step 0: loss 1.4999802112579346
step 1: loss 0.5064970850944519
step 100: loss 0.11254584789276123
step 499: loss 0.030913986265659332
rolling window:  3
step 0: loss 0.14402541518211365
step 1: loss 0.22435027360916138
step 100: loss 0.029584594070911407
step 499: loss 0.004174630623310804
rolling window:  4
step 0: loss 12.0000638961792
step 1: loss 12.407329559326172
step 100: loss 0.0552961491048336
step 499: loss 8.779268682701513e-06
rolling window:  5
step 0: loss 0.16224625706672668
step 1: loss 0.08104564249515533
step 100: loss 1.3870835573470686e-05
step 499: loss 4.903565054004133e-11
rolling windo

In [3]:
abs(y.iloc[-12:].values - np.array(y_hat_umlp)).mean()

15.922597908539892

# Mini-Batch

In [4]:
# read processed data
df = pd.read_csv("../data/processed/tidy_df.csv", index_col=[0, 1, 2])

# empty list for dataframes
y_test_list = []
y_hat_umlp = []

i = df.index.get_level_values(0).unique()[0]
print(i)

# y : "EPS"
y = df.loc[pd.IndexSlice[i, :, :], "EPS"]

# x, exogenous regressors : 'INV', 'AR', 'CAPX', 'GM', 'SA', 'ETR', 'LF'
#     x = df.loc[pd.IndexSlice[i, :, :], ['INV', 'AR', 'CAPX', 'GM', 'SA', 'ETR', 'LF']]

# Unlike statsmodel SARIMA package, NN needs to prepare lagged inputs manually if needed.
# y_lag and x_lag (lag 4 for now)
num_lag = 4
y_lag = lag(y, num_lag, drop_nan=False, reset_index=False)
#     x_lag = lag(x, num_lag, drop_nan=False, reset_index=False)

# Redefine data name as target (y) and feature (y_lag) (explanatory variable, predictor)
target = y
feature = y_lag

# save simple test data series
_, target_test_dataset = train_test_split(target, ratio=(4,1))
_, feature_test_dataset = train_test_split(feature, ratio=(4,1))

# drop nan caused by lag()
feature = feature.dropna(axis=0)
target = target[feature.index]

# setting torch
dtype = torch.float # double float problem in layer 
device = torch.device("cpu")

# Make data to torch.tensor
target = torch.tensor(target.values, dtype=dtype)
feature = torch.tensor(feature.values, dtype=dtype)
target_test_dataset = torch.tensor(target_test_dataset.values, dtype=dtype)
feature_test_dataset = torch.tensor(feature_test_dataset.values, dtype=dtype)

# rolling window data preparation

### ! Hyper-Parameter ! ##########################################################
# all period: 48, train 36, test 12
test_window = len(target_test_dataset)
print("test window: ", test_window)

train_window = len(target) - test_window
print("train window: ", train_window)
##################################################################################

train_dataset = TimeSeriesDataset(feature, target, train_window)
print("len of train dataset: ", len(train_dataset))
#     len(train_dataset) == len(target) - train_window = 48 - 36 = 12 == test_window
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)

np.random.seed(0)
torch.manual_seed(0)

### ! Hyper-Parameter ! ##########################################################
num_epochs = 500
learning_rate = 1e-2
input_units = 1
hidden_units = 512
# num_layers = 1
output_units = 1
# Optimizer
batch_size = 2

model_name = 'umlp' + '_hid' + str(hidden_units) + '_lr' + str(learning_rate) + '_epo' + str(num_epochs)
##################################################################################

# load rolling window data flow
for num_window, (feature_train_all_batch, target_train_all_batch) in enumerate(train_loader):
    print("WINDOW: ", num_window)
#     print(feature_train_all_batch.size(), target_train_all_batch.size())
    feature_train_all_batch = feature_train_all_batch[0] # extract single batch
    target_train_all_batch = target_train_all_batch[0] # extract single batch

#     #####Init the Model #######################
#     mlp = MLP(input_features=feature_train.size()[1], hidden_units=hidden_units, output_units=output_units)
#     ##### Set Criterion Optimzer and scheduler ####################
#     criterion = torch.nn.MSELoss(reduction="mean")
#     optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate) # link to mlp parameters (lr should be 1e-2)
    
    #(only first window)
    if num_window == 0:
        #####Init the Model #######################
        mlp = MLP(input_features=feature_train.size()[1], hidden_units=hidden_units, output_units=output_units)
        ##### Set Criterion Optimzer and scheduler ####################
        criterion = torch.nn.MSELoss(reduction="mean")
        optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate) # link to mlp parameters (lr should be 1e-2)
    else:
        pass
    
    # mini-batch loader
    batch_dataset = TimeSeriesDataset(feature_train_all_batch, target_train_all_batch, train_window=None)
    minibatch_loader = torch.utils.data.DataLoader(batch_dataset, batch_size=batch_size, shuffle=False)

    # Train the model: Learning iteration
    # use pre window trained model's weight as initial weight (continue training)
    #             print("initial weight: ", mlp.state_dict()['hidden.weight'][0])

    for epoch in range(num_epochs):
        for num_mini_batch, (feature_train_mini_batch, target_train_mini_batch) in enumerate(minibatch_loader):    
            # Forward pass
            target_pred = mlp(feature_train_mini_batch)
            # let y_pred be the same size as y
            target_pred = target_pred.squeeze(1)

            # Compute loss
            loss = criterion(target_pred, target_train_mini_batch) # link to mlp output

            # Zero gradients, perform backward pass, and update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (epoch == 0) | (epoch == 1) | (epoch == 100) | (epoch == 499):
            print(f"epoch {epoch}: loss {loss.item()}")

    # Save the trained model
#     PATH = '../../assets/trained_models/univariate/mlp/' + model_name + "_" + i + "_" + "win" + str(num_window) + '.pth'
#     torch.save(mlp.state_dict(), PATH)
    # use the existing trained model and continute training next window.
    #             print("inherit weight: ", mlp.state_dict()['hidden.weight'][0])

    # predict y_hat (target_hat) <- 良くないかも、with torch_nograd() と model.eval()
    with torch.no_grad():
        target_test = target_test_dataset[num_window]
        feature_test = feature_test_dataset[num_window]
        y_hat_umlp.append(mlp(feature_test).squeeze().detach().numpy())
    #                 print(feature_test)
    #                 print(target_test)
    #                 print(y_hat_umlp[-1])

あらた
test window:  12
train window:  36
len of train dataset:  12
WINDOW:  0
epoch 0: loss 6991.52685546875
epoch 1: loss 5212.80419921875
epoch 100: loss 63.719154357910156
epoch 499: loss 20.02129554748535
WINDOW:  1
epoch 0: loss 49.18145751953125
epoch 1: loss 23.821441650390625
epoch 100: loss 110.38540649414062
epoch 499: loss 19.06254768371582
WINDOW:  2
epoch 0: loss 136.7188262939453
epoch 1: loss 63.97145080566406
epoch 100: loss 49.73845291137695
epoch 499: loss 6.5564351081848145
WINDOW:  3
epoch 0: loss 61.90725326538086
epoch 1: loss 46.753807067871094
epoch 100: loss 13.831600189208984
epoch 499: loss 31.53185272216797
WINDOW:  4
epoch 0: loss 328.1629638671875
epoch 1: loss 142.90606689453125
epoch 100: loss 89.15724182128906
epoch 499: loss 19.888721466064453
WINDOW:  5
epoch 0: loss 28.871501922607422
epoch 1: loss 33.40175247192383
epoch 100: loss 8.478342056274414
epoch 499: loss 79.77935791015625
WINDOW:  6
epoch 0: loss 137.2894287109375
epoch 1: loss 126.727828979

In [5]:
abs(y.iloc[-12:].values - np.array(y_hat_umlp)).mean()

19.519032412730972